# 🛰️ SNUNet Change Detection - Kaggle Training

**Setup:**
1. Settings → Accelerator → **GPU T4 x2**
2. Add Data → Upload **LEVIR-CD-kaggle.zip** as dataset

In [ ]:
# === SETUP ===
import os, torch, torch.nn as nn, torch.nn.functional as F, numpy as np, zipfile
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cpu': print("⚠️ Enable GPU: Settings → Accelerator → GPU T4")

In [ ]:
# === EXTRACT DATASET ===
DATA_ROOT = '/kaggle/working/data'

# Find and extract zip from input
if not os.path.exists(os.path.join(DATA_ROOT, 'train', 'A')):
    zip_path = None
    if os.path.exists('/kaggle/input'):
        for d in os.listdir('/kaggle/input'):
            for f in os.listdir(os.path.join('/kaggle/input', d)):
                if f.endswith('.zip'):
                    zip_path = os.path.join('/kaggle/input', d, f)
                    break
            # Also check if data is already extracted in input
            check_path = os.path.join('/kaggle/input', d)
            if os.path.exists(os.path.join(check_path, 'train', 'A')):
                DATA_ROOT = check_path
                print(f"✅ Found extracted data at: {DATA_ROOT}")
                break
    
    if zip_path and not os.path.exists(os.path.join(DATA_ROOT, 'train', 'A')):
        print(f"Extracting {zip_path}...")
        os.makedirs(DATA_ROOT, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(DATA_ROOT)
        print("✅ Extracted!")
    elif not os.path.exists(os.path.join(DATA_ROOT, 'train', 'A')):
        print("❌ No dataset found. Upload LEVIR-CD-kaggle.zip to Add Data.")
        raise FileNotFoundError("Dataset not found")

# Find nested folder if exists
for sub in ['', 'LEVIR-CD-patches', 'LEVIR-CD']:
    check = os.path.join(DATA_ROOT, sub) if sub else DATA_ROOT
    if os.path.exists(os.path.join(check, 'train', 'A')):
        DATA_ROOT = check
        break

print(f"Data: {DATA_ROOT}")
for s in ['train','val','test']:
    p = os.path.join(DATA_ROOT, s, 'A')
    if os.path.exists(p): print(f"  {s}: {len(os.listdir(p))}")

In [ ]:
# === MODEL ===
class CB(nn.Module):
    def __init__(self,i,o): super().__init__(); self.c=nn.Sequential(nn.Conv2d(i,o,3,padding=1),nn.BatchNorm2d(o),nn.ReLU(True),nn.Conv2d(o,o,3,padding=1),nn.BatchNorm2d(o),nn.ReLU(True))
    def forward(self,x): return self.c(x)
class At(nn.Module):
    def __init__(self,c): super().__init__(); self.a=nn.Sequential(nn.AdaptiveAvgPool2d(1),nn.Conv2d(c,c//8,1),nn.ReLU(),nn.Conv2d(c//8,c,1),nn.Sigmoid())
    def forward(self,x): return x*self.a(x)
class SNUNet(nn.Module):
    def __init__(self,c=32):
        super().__init__()
        self.e1,self.e2,self.e3,self.e4,self.e5=CB(3,c),CB(c,c*2),CB(c*2,c*4),CB(c*4,c*8),CB(c*8,c*16)
        self.pool=nn.MaxPool2d(2)
        self.u4,self.d4,self.a4=nn.ConvTranspose2d(c*16,c*8,2,2),CB(c*24,c*8),At(c*8)
        self.u3,self.d3,self.a3=nn.ConvTranspose2d(c*8,c*4,2,2),CB(c*12,c*4),At(c*4)
        self.u2,self.d2,self.a2=nn.ConvTranspose2d(c*4,c*2,2,2),CB(c*6,c*2),At(c*2)
        self.u1,self.d1,self.a1=nn.ConvTranspose2d(c*2,c,2,2),CB(c*3,c),At(c)
        self.out=nn.Conv2d(c,1,1)
    def forward(self,x1,x2):
        e11,e12=self.e1(x1),self.e1(x2)
        e21,e22=self.e2(self.pool(e11)),self.e2(self.pool(e12))
        e31,e32=self.e3(self.pool(e21)),self.e3(self.pool(e22))
        e41,e42=self.e4(self.pool(e31)),self.e4(self.pool(e32))
        e51,e52=self.e5(self.pool(e41)),self.e5(self.pool(e42))
        d=torch.abs(e51-e52)
        d=self.a4(self.d4(torch.cat([self.u4(d),e41,e42],1)))
        d=self.a3(self.d3(torch.cat([self.u3(d),e31,e32],1)))
        d=self.a2(self.d2(torch.cat([self.u2(d),e21,e22],1)))
        d=self.a1(self.d1(torch.cat([self.u1(d),e11,e12],1)))
        return self.out(d)
model=SNUNet(32).to(device)
print(f"✅ Model: {sum(p.numel() for p in model.parameters()):,} params")

In [ ]:
# === DATASET ===
class CD(Dataset):
    def __init__(self,r,s): self.r,self.s=r,s; self.f=sorted([x for x in os.listdir(os.path.join(r,s,'A')) if x.endswith('.png')])
    def __len__(self): return len(self.f)
    def __getitem__(self,i):
        n=self.f[i]
        i1=np.array(Image.open(os.path.join(self.r,self.s,'A',n)).convert('RGB').resize((256,256)))
        i2=np.array(Image.open(os.path.join(self.r,self.s,'B',n)).convert('RGB').resize((256,256)))
        lb=np.array(Image.open(os.path.join(self.r,self.s,'label',n)).convert('L').resize((256,256)))
        return torch.from_numpy(i1).permute(2,0,1).float()/255,torch.from_numpy(i2).permute(2,0,1).float()/255,(torch.from_numpy(lb).float()/255>0.5).float().unsqueeze(0)

train_ds=CD(DATA_ROOT,'train')
val_s='val' if os.path.exists(os.path.join(DATA_ROOT,'val','A')) else 'test'
val_ds=CD(DATA_ROOT,val_s)
train_ld=DataLoader(train_ds,8,shuffle=True,num_workers=2,pin_memory=True)
val_ld=DataLoader(val_ds,16,num_workers=2)
print(f"✅ Train:{len(train_ds)} Val:{len(val_ds)}")

In [ ]:
# === TRAINING ===
import csv
def loss_fn(p,t): return F.binary_cross_entropy_with_logits(p,t)+1-(2*(torch.sigmoid(p)*t).sum()+1)/(torch.sigmoid(p).sum()+t.sum()+1)
def f1(p,t): p=(torch.sigmoid(p)>0.5).float(); tp=(p*t).sum(); return (2*tp/(2*tp+(p*(1-t)).sum()+((1-p)*t).sum()+1e-8)).item()

opt=torch.optim.AdamW(model.parameters(),1e-4,weight_decay=1e-4)
sched=torch.optim.lr_scheduler.CosineAnnealingLR(opt,100)
os.makedirs('/kaggle/working/ckpt',exist_ok=True)
log=open('/kaggle/working/ckpt/log.csv','w'); w=csv.writer(log); w.writerow(['ep','tr','va','f1'])

best,pat=0,0
for ep in range(100):
    model.train(); tr=0
    for x1,x2,y in tqdm(train_ld,desc=f"Ep{ep+1}",leave=False):
        x1,x2,y=x1.to(device),x2.to(device),y.to(device)
        opt.zero_grad(); out=model(x1,x2); L=loss_fn(out,y); L.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),1); opt.step(); tr+=L.item()
    tr/=len(train_ld)
    model.eval(); va,f1s=0,0
    with torch.no_grad():
        for x1,x2,y in val_ld:
            x1,x2,y=x1.to(device),x2.to(device),y.to(device); out=model(x1,x2); va+=loss_fn(out,y).item(); f1s+=f1(out,y)
    va/=len(val_ld); f1s/=len(val_ld); sched.step()
    print(f"Ep{ep+1:3d}|TrL:{tr:.4f}|VaL:{va:.4f}|F1:{f1s:.4f}")
    w.writerow([ep+1,tr,va,f1s]); log.flush()
    if f1s>best: best,pat=f1s,0; torch.save(model.state_dict(),'/kaggle/working/ckpt/best.pth'); print(f"  >>Saved F1={f1s:.4f}")
    else: pat+=1
    if pat>=20: print("Early stop!"); break
log.close(); print(f"\n✅ Best F1: {best:.4f}")

In [ ]:
# === RESULTS ===
import matplotlib.pyplot as plt, pandas as pd
model.load_state_dict(torch.load('/kaggle/working/ckpt/best.pth')); model.eval()
df=pd.read_csv('/kaggle/working/ckpt/log.csv')
fig,ax=plt.subplots(1,2,figsize=(12,4))
ax[0].plot(df['ep'],df['tr'],label='Train'); ax[0].plot(df['ep'],df['va'],label='Val'); ax[0].legend(); ax[0].set_title('Loss')
ax[1].plot(df['ep'],df['f1'],'g-'); ax[1].axhline(0.85,color='r',ls='--'); ax[1].set_title(f'F1 (Best:{df.f1.max():.4f})')
plt.tight_layout(); plt.savefig('/kaggle/working/curves.png'); plt.show()

fig,ax=plt.subplots(3,4,figsize=(12,9))
for r in range(3):
    x1,x2,y=val_ds[r*5]
    with torch.no_grad(): p=(torch.sigmoid(model(x1.unsqueeze(0).to(device),x2.unsqueeze(0).to(device)))>0.5).cpu().squeeze().numpy()
    ax[r,0].imshow(x1.permute(1,2,0)); ax[r,1].imshow(x2.permute(1,2,0)); ax[r,2].imshow(y.squeeze(),cmap='gray'); ax[r,3].imshow(p,cmap='gray')
    for c in range(4): ax[r,c].axis('off')
ax[0,0].set_title('T1'); ax[0,1].set_title('T2'); ax[0,2].set_title('GT'); ax[0,3].set_title('Pred')
plt.tight_layout(); plt.savefig('/kaggle/working/preds.png'); plt.show()
print("✅ Done!")